# This code is used to calculate steric height on Pleiades

In [3]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from xmitgcm import llcreader
import zarr

In [4]:
model = llcreader.PleiadesLLC4320Model()
ds = model.get_dataset(varnames=['Eta','PhiBot'])

In [3]:
#Let's write the relevant atmospheric pressure to zarr
fp = np.memmap('/nobackup/dmenemen/forcing/ECMWF_operational/EOG_pres_tide_2011',
               dtype='>f4',mode='r',shape=(8760,1280,2560))

lon = np.cumsum(np.asarray([0] + [0.140625] * 2559))
lat = -89.8924 + np.cumsum(np.asarray([0, .1394, .14018, .14039, .1404695, .140496, 
                             .1405145, .1405275, .1405375, .1405455,
                             .140552, .1405575, .140562, .1405655,
                             .140568, .1405695] +  [.14057] * 1249 + 
                    [.1405695, .140568, .1405655, .140562,
                     .1405575, .140552, .1405455,.1405375,
                     .1405275, .1405145, .140496, .1404695,
                     .14039, .14018, .1394]))
lon[lon>180] = lon[lon>180]-360

t=0
newarray = fp[-2640+t,:,:]
newarray = np.expand_dims(newarray,axis=0)
time = np.expand_dims(np.asarray(ds.time.isel(time=t).values),0)
pres_xr = xr.DataArray(newarray, dims=['time','lat','lon'],coords=dict(lon=lon,lat=lat))#time=time,
surface_pressure = (pres_xr.roll(lon=200)).sel(lat=slice(-58,-26)).isel(lon=slice(88,455))
surface_pressure.chunk({'lat':-1}).to_dataset(name='Pressure').to_zarr('atm_pressure.zarr')
for t in range(1,2000):
    newarray = fp[-2640+t,:,:]
    newarray = np.expand_dims(newarray,axis=0)
    time = np.expand_dims(np.asarray(ds.time.isel(time=t).values),0)
    pres_xr = xr.DataArray(newarray, dims=['time','lat','lon'],coords=dict(lon=lon,lat=lat))#time=time,
    surface_pressure = (pres_xr.roll(lon=200)).sel(lat=slice(-58,-26)).isel(lon=slice(88,455))
    surface_pressure.chunk({'lat':-1}).to_dataset(name='Pressure').to_zarr('atm_pressure.zarr',append_dim='time')

/nobackup/csjone15/miniconda3/envs/pangeo-0ac8885/lib/python3.8/site-packages/xarray/core/dataarray.py:3267: FutureWarning: roll_coords will be set to False in the future. Explicitly set roll_coords to silence warning.
  ds = self._to_temp_dataset().roll(


In [3]:
surface_pressure=xr.open_zarr('atm_pressure.zarr').Pressure

In [5]:
import xesmf as xe

In [6]:
face_lat = ds.Eta.isel(time=0,face=1,j=slice(0,2160),i=slice(1080,1080+2160)).YC.mean('i')
face_lon = ds.Eta.isel(time=0,face=1,j=slice(0,2160),i=slice(1080,1080+2160)).XC.mean('j')

In [7]:
ds_out = xr.Dataset({'lat': (['lat'], face_lat.values),
                     'lon': (['lon'], face_lon.values),
                    }
                   )

In [8]:
surface_pressure_rename = surface_pressure.rename({'lon':'longitude','lat':'latitude'})

In [9]:
ds_out_rename = ds_out.rename({'lon':'longitude','lat':'latitude'})

In [10]:
regridder = xe.Regridder(surface_pressure_rename.T.isel(time=0), ds_out_rename, 'bilinear')

/nobackup/csjone15/miniconda3/envs/pangeo-0ac8885/lib/python3.8/site-packages/dask/array/core.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  o = func(*args, **kwargs)


In [11]:
dr_out = regridder(surface_pressure_rename.isel(time=[0]).load())
dr_ij = dr_out.drop_vars(['latitude','longitude']).rename({'latitude':'j','longitude':'i'})
dr_ij.chunk({'i':-1,'j':-1}).to_dataset(name='Pressure').to_zarr('regridded_pressure.zarr')

for t in range(1,2000):
    dr_out = regridder(surface_pressure_rename.isel(time=[t]).load())
    dr_ij = dr_out.drop_vars(['latitude','longitude']).rename({'latitude':'j','longitude':'i'})
    dr_ij.chunk({'i':-1,'j':-1}).to_dataset(name='Pressure').to_zarr('regridded_pressure.zarr', append_dim='time')

In [5]:
dr_ij = xr.open_zarr('regridded_pressure.zarr').Pressure

In [4]:
mean_phibot = xr.open_zarr('mean_phibot.zarr')
phibot_mr = ds.PhiBot.isel(face=1,j=slice(0,2160),i=slice(1080,1080+2160)) - mean_phibot.PhiBot

In [5]:
mean_eta = xr.open_zarr('mean_eta.zarr')
eta_mr = ds.Eta.isel(face=1,j=slice(0,2160),i=slice(1080,1080+2160)
                          ) - mean_eta.Eta

In [6]:
mean_pres = xr.open_zarr('mean_atm_pressure.zarr')
pres_mr = dr_ij - mean_pres.pressure.load()
#pres_mr = pres_mr#.assign_coords(time=eta_mr.time.isel(time=slice(2640-2640,2640-1440)))

In [6]:
steric_est = (ds.Eta.isel(face=1,j=slice(0,2160),i=slice(1080,1080+2160)
                          ).isel(time=slice(2640-2640,2640-721))
              +dr_ij.isel(time=slice(2640-2640,2640-721))/9.81/1027.5
 -ds.PhiBot.isel(face=1,j=slice(0,2160),i=slice(1080,1080+2160)).isel(time=slice(2640-2640,2640-721))/9.81)

In [11]:
steric_replace_missing = steric_est
steric_est_chosen_loc = steric_est.isel(j=1000,i=1000)    
steric_replace_missing = steric_replace_missing.where(steric_est_chosen_loc<-20)
steric_replace_missing = steric_replace_missing.ffill('time')

In [15]:
steric_replace_missing.to_dataset(name='steric').to_zarr('steric_inc_mean.zarr')

# Mean atmospheric pressure is calculated below


In [6]:
fp = np.memmap('/nobackup/dmenemen/forcing/ECMWF_operational/EOG_pres_tide_2011',
               dtype='>f4',mode='r',shape=(8760,1280,2560))

In [ ]:
#find mean atmospheric pressure in this region 2011
counter = 0
atm_slice=np.zeros((1280, 2560))
for t in range(0,8760):
    atm_slice = atm_slice + fp[t*3,:,:]
    if (np.mod(t,100)==0):
        print(t)
    counter=counter+1
mean_pressure = atm_slice/counter

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900


IndexError: index 8760 is out of bounds for axis 0 with size 8760

In [9]:
fp = np.memmap('/nobackup/dmenemen/forcing/ECMWF_operational/EOG_pres_tide_2012',
               dtype='>f4',mode='r',shape=(8760,1280,2560))

In [10]:
#find mean atmospheric pressure in this region 2012
for t in range(0,3000):
    atm_slice = atm_slice + fp[t*3,:,:]
    if (np.mod(t,100)==0):
        print(t)
    counter=counter+1
mean_pressure = atm_slice/counter

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900


IndexError: index 8760 is out of bounds for axis 0 with size 8760

In [31]:
fp = np.memmap('/nobackup/dmenemen/forcing/ECMWF_operational/EOG_pres_tide_2013',
               dtype='>f4',mode='r',shape=(5826,1280,2560))

In [32]:
#find mean atmospheric pressure in this region 2013
for t in range(0,3000):
    atm_slice = atm_slice + fp[t*3,:,:]
    if (np.mod(t,100)==0):
        print(t)
    counter=counter+1
mean_pressure = atm_slice/counter

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


IndexError: index 5826 is out of bounds for axis 0 with size 5826

In [34]:
mean_pressure = atm_slice/counter

In [43]:
pres_xr = xr.DataArray(mean_pressure, dims=['lat','lon'],coords=dict(lon=lon,lat=lat))

In [44]:
surface_pressure = (pres_xr.roll(lon=200)).sel(lat=slice(-58,-26)).isel(lon=slice(88,455))

/nobackup/csjone15/miniconda3/envs/pangeo-0ac8885/lib/python3.8/site-packages/xarray/core/dataarray.py:3267: FutureWarning: roll_coords will be set to False in the future. Explicitly set roll_coords to silence warning.
  ds = self._to_temp_dataset().roll(


In [51]:
surface_pressure_rename = surface_pressure.rename({'lon':'longitude','lat':'latitude'})

In [49]:
ds_out_rename = ds_out.rename({'lon':'longitude','lat':'latitude'})

In [52]:
regridder = xe.Regridder(surface_pressure_rename, ds_out_rename, 'bilinear')

/nobackup/csjone15/miniconda3/envs/pangeo-0ac8885/lib/python3.8/site-packages/xarray/core/dataarray.py:789: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self.data


In [53]:
dr_out = regridder(surface_pressure_rename)

/nobackup/csjone15/miniconda3/envs/pangeo-0ac8885/lib/python3.8/site-packages/xesmf/smm.py:129: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')


In [54]:
dr_ij = dr_out.drop_vars(['latitude','longitude']).rename({'latitude':'j','longitude':'i'})

In [58]:
dr_ij.to_dataset(name='pressure').to_zarr('mean_atm_pressure.zarr')